# Prepare the dataset
The sample code of chapter 5 relies on the availability of the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). This dataset contains 60.000 hand-written digits and 10.000 test samples. This notebook contains everythin you need to download the dataset.

Please run all the cells through the menu item `Cell` > `Run all`

## Utility functions
This notebook contains a number of utility functions that download and process the MNIST dataset. The first utility function is the `download_data` function. This function will retrieve the raw images for the MNIST dataset as a numpy array.

In [1]:
from urllib.request import urlretrieve
import gzip
import struct
import numpy as np
from scipy.misc import imsave
import os
import imageio

In [2]:
def load_data(url, image_count):
    print('Downloading image data from {}'.format(url))
    
    filename, h = urlretrieve(url, './.data_file')

    try:
        with gzip.open(filename) as gz:
            n = struct.unpack('I', gz.read(4))
            
            # The input file contains a magic number to validate the contents of the file.
            # This check makes sure that the file is valid.
            if n[0] != 0x3080000:
                raise Exception('Invalid file: unexpected magic number.')
            
            # Next to the magic number is the number of records in the file.
            # This number should match with the image count parameter.
            n = struct.unpack('>I', gz.read(4))[0]
            
            if n != image_count:
                raise Exception('Invalid file: expected {0} entries.'.format(image_count))
            
            crow = struct.unpack('>I', gz.read(4))[0]
            ccol = struct.unpack('>I', gz.read(4))[0]
            
            # Make sure that we have the right image format. It should be 28x28.
            if crow != 28 or ccol != 28:
                raise Exception('Invalid file: expected 28 rows/cols per image.')
            
            # We'll reshape and write it to disk as png images later.
            image_data = np.frombuffer(gz.read(image_count * crow * ccol), dtype = np.uint8)
    finally:
        os.remove(filename)
        
    return image_data.reshape((image_count, crow * ccol))

The second utility function downloads the labels file for the dataset. The labels are stored as a tarball and need the same extraction logic as the data file. Except this time we're reshaping the data as a 1D array.

In [3]:
def load_labels(url, image_count):
    print('Downloading label data from {}'.format(url))
    
    filename, h = urlretrieve(url, './.label_file')

    try:
        with gzip.open(filename) as gz:
            n = struct.unpack('I', gz.read(4))
            
            # The input file contains a magic number to validate the contents of the file.
            # This check makes sure that the file is valid.
            if n[0] != 0x1080000:
                raise Exception('Invalid file: unexpected magic number.')
                
            # Next to the magic number is the number of records in the file.
            # This number should match with the image count parameter.
            n = struct.unpack('>I', gz.read(4))
            
            if n[0] != image_count:
                raise Exception('Invalid file: expected {0} rows.'.format(image_count))
                
            # Read all the labels from disk into an array.
            res = np.frombuffer(gz.read(image_count), dtype = np.uint8)
    finally:
        os.remove(filename)
    return res.reshape((image_count, 1))

To make downloading more straightforward we'll use the `download_dataset` function. This function uses the download functions for the data and the labels to get the data. It then stores the images on disk and creates a mapping file for the dataset. 

In [4]:
def download_dataset(output_folder, data_url, labels_url, image_count):
    os.makedirs(output_folder, exist_ok=True)
    
    data = load_data(data_url, image_count)
    labels = load_labels(labels_url, image_count)
    
    with open(os.path.join(output_folder, 'mapping.bin'),'w') as mapping_file:
        for index in range(0, image_count):
            print('Storing image {} of {}'.format(index + 1, image_count))
            
            filename = '{:0>10}.png'.format(index)
            
            imageio.imwrite(os.path.join(output_folder, filename), data[index].reshape(28,28))
            mapping_file.write('{}/{}\t{}\n'.format(output_folder, filename, labels[index][0]))

## Downloading the dataset
This section contains the logic to actually download the training and validation sets for the MNIST dataset.
Be aware that this takes a long time on an average computer. The output is stored in `mnist_train` and `mnist_validate` folders.

In [5]:
train_image_url = 'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz'
train_label_url = 'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz'
train_samples = 60000

download_dataset('mnist_train', train_image_url, train_label_url, train_samples)

Storing image 1 of 60000
Storing image 2 of 60000
Storing image 3 of 60000
Storing image 4 of 60000
Storing image 5 of 60000
Storing image 6 of 60000
Storing image 7 of 60000
Storing image 8 of 60000
Storing image 9 of 60000
Storing image 10 of 60000
Storing image 11 of 60000
Storing image 12 of 60000
Storing image 13 of 60000
Storing image 14 of 60000
Storing image 15 of 60000
Storing image 16 of 60000
Storing image 17 of 60000
Storing image 18 of 60000
Storing image 19 of 60000
Storing image 20 of 60000
Storing image 21 of 60000
Storing image 22 of 60000
Storing image 23 of 60000
Storing image 24 of 60000
Storing image 25 of 60000
Storing image 26 of 60000
Storing image 27 of 60000
Storing image 28 of 60000
Storing image 29 of 60000
Storing image 30 of 60000
Storing image 31 of 60000
Storing image 32 of 60000
Storing image 33 of 60000
Storing image 34 of 60000
Storing image 35 of 60000
Storing image 36 of 60000
Storing image 37 of 60000
Storing image 38 of 60000
Storing image 39 of 6

Storing image 351 of 60000
Storing image 352 of 60000
Storing image 353 of 60000
Storing image 354 of 60000
Storing image 355 of 60000
Storing image 356 of 60000
Storing image 357 of 60000
Storing image 358 of 60000
Storing image 359 of 60000
Storing image 360 of 60000
Storing image 361 of 60000
Storing image 362 of 60000
Storing image 363 of 60000
Storing image 364 of 60000
Storing image 365 of 60000
Storing image 366 of 60000
Storing image 367 of 60000
Storing image 368 of 60000
Storing image 369 of 60000
Storing image 370 of 60000
Storing image 371 of 60000
Storing image 372 of 60000
Storing image 373 of 60000
Storing image 374 of 60000
Storing image 375 of 60000
Storing image 376 of 60000
Storing image 377 of 60000
Storing image 378 of 60000
Storing image 379 of 60000
Storing image 380 of 60000
Storing image 381 of 60000
Storing image 382 of 60000
Storing image 383 of 60000
Storing image 384 of 60000
Storing image 385 of 60000
Storing image 386 of 60000
Storing image 387 of 60000
S

Storing image 782 of 60000
Storing image 783 of 60000
Storing image 784 of 60000
Storing image 785 of 60000
Storing image 786 of 60000
Storing image 787 of 60000
Storing image 788 of 60000
Storing image 789 of 60000
Storing image 790 of 60000
Storing image 791 of 60000
Storing image 792 of 60000
Storing image 793 of 60000
Storing image 794 of 60000
Storing image 795 of 60000
Storing image 796 of 60000
Storing image 797 of 60000
Storing image 798 of 60000
Storing image 799 of 60000
Storing image 800 of 60000
Storing image 801 of 60000
Storing image 802 of 60000
Storing image 803 of 60000
Storing image 804 of 60000
Storing image 805 of 60000
Storing image 806 of 60000
Storing image 807 of 60000
Storing image 808 of 60000
Storing image 809 of 60000
Storing image 810 of 60000
Storing image 811 of 60000
Storing image 812 of 60000
Storing image 813 of 60000
Storing image 814 of 60000
Storing image 815 of 60000
Storing image 816 of 60000
Storing image 817 of 60000
Storing image 818 of 60000
S

Storing image 1151 of 60000
Storing image 1152 of 60000
Storing image 1153 of 60000
Storing image 1154 of 60000
Storing image 1155 of 60000
Storing image 1156 of 60000
Storing image 1157 of 60000
Storing image 1158 of 60000
Storing image 1159 of 60000
Storing image 1160 of 60000
Storing image 1161 of 60000
Storing image 1162 of 60000
Storing image 1163 of 60000
Storing image 1164 of 60000
Storing image 1165 of 60000
Storing image 1166 of 60000
Storing image 1167 of 60000
Storing image 1168 of 60000
Storing image 1169 of 60000
Storing image 1170 of 60000
Storing image 1171 of 60000
Storing image 1172 of 60000
Storing image 1173 of 60000
Storing image 1174 of 60000
Storing image 1175 of 60000
Storing image 1176 of 60000
Storing image 1177 of 60000
Storing image 1178 of 60000
Storing image 1179 of 60000
Storing image 1180 of 60000
Storing image 1181 of 60000
Storing image 1182 of 60000
Storing image 1183 of 60000
Storing image 1184 of 60000
Storing image 1185 of 60000
Storing image 1186 o

Storing image 1532 of 60000
Storing image 1533 of 60000
Storing image 1534 of 60000
Storing image 1535 of 60000
Storing image 1536 of 60000
Storing image 1537 of 60000
Storing image 1538 of 60000
Storing image 1539 of 60000
Storing image 1540 of 60000
Storing image 1541 of 60000
Storing image 1542 of 60000
Storing image 1543 of 60000
Storing image 1544 of 60000
Storing image 1545 of 60000
Storing image 1546 of 60000
Storing image 1547 of 60000
Storing image 1548 of 60000
Storing image 1549 of 60000
Storing image 1550 of 60000
Storing image 1551 of 60000
Storing image 1552 of 60000
Storing image 1553 of 60000
Storing image 1554 of 60000
Storing image 1555 of 60000
Storing image 1556 of 60000
Storing image 1557 of 60000
Storing image 1558 of 60000
Storing image 1559 of 60000
Storing image 1560 of 60000
Storing image 1561 of 60000
Storing image 1562 of 60000
Storing image 1563 of 60000
Storing image 1564 of 60000
Storing image 1565 of 60000
Storing image 1566 of 60000
Storing image 1567 o

...

Storing image 58451 of 60000
Storing image 58452 of 60000
Storing image 58453 of 60000
Storing image 58454 of 60000
Storing image 58455 of 60000
Storing image 58456 of 60000
Storing image 58457 of 60000
Storing image 58458 of 60000
Storing image 58459 of 60000
Storing image 58460 of 60000
Storing image 58461 of 60000
Storing image 58462 of 60000
Storing image 58463 of 60000
Storing image 58464 of 60000
Storing image 58465 of 60000
Storing image 58466 of 60000
Storing image 58467 of 60000
Storing image 58468 of 60000
Storing image 58469 of 60000
Storing image 58470 of 60000
Storing image 58471 of 60000
Storing image 58472 of 60000
Storing image 58473 of 60000
Storing image 58474 of 60000
Storing image 58475 of 60000
Storing image 58476 of 60000
Storing image 58477 of 60000
Storing image 58478 of 60000
Storing image 58479 of 60000
Storing image 58480 of 60000
Storing image 58481 of 60000
Storing image 58482 of 60000
Storing image 58483 of 60000
Storing image 58484 of 60000
Storing image 

Storing image 58854 of 60000
Storing image 58855 of 60000
Storing image 58856 of 60000
Storing image 58857 of 60000
Storing image 58858 of 60000
Storing image 58859 of 60000
Storing image 58860 of 60000
Storing image 58861 of 60000
Storing image 58862 of 60000
Storing image 58863 of 60000
Storing image 58864 of 60000
Storing image 58865 of 60000
Storing image 58866 of 60000
Storing image 58867 of 60000
Storing image 58868 of 60000
Storing image 58869 of 60000
Storing image 58870 of 60000
Storing image 58871 of 60000
Storing image 58872 of 60000
Storing image 58873 of 60000
Storing image 58874 of 60000
Storing image 58875 of 60000
Storing image 58876 of 60000
Storing image 58877 of 60000
Storing image 58878 of 60000
Storing image 58879 of 60000
Storing image 58880 of 60000
Storing image 58881 of 60000
Storing image 58882 of 60000
Storing image 58883 of 60000
Storing image 58884 of 60000
Storing image 58885 of 60000
Storing image 58886 of 60000
Storing image 58887 of 60000
Storing image 

Storing image 59231 of 60000
Storing image 59232 of 60000
Storing image 59233 of 60000
Storing image 59234 of 60000
Storing image 59235 of 60000
Storing image 59236 of 60000
Storing image 59237 of 60000
Storing image 59238 of 60000
Storing image 59239 of 60000
Storing image 59240 of 60000
Storing image 59241 of 60000
Storing image 59242 of 60000
Storing image 59243 of 60000
Storing image 59244 of 60000
Storing image 59245 of 60000
Storing image 59246 of 60000
Storing image 59247 of 60000
Storing image 59248 of 60000
Storing image 59249 of 60000
Storing image 59250 of 60000
Storing image 59251 of 60000
Storing image 59252 of 60000
Storing image 59253 of 60000
Storing image 59254 of 60000
Storing image 59255 of 60000
Storing image 59256 of 60000
Storing image 59257 of 60000
Storing image 59258 of 60000
Storing image 59259 of 60000
Storing image 59260 of 60000
Storing image 59261 of 60000
Storing image 59262 of 60000
Storing image 59263 of 60000
Storing image 59264 of 60000
Storing image 

Storing image 59530 of 60000
Storing image 59531 of 60000
Storing image 59532 of 60000
Storing image 59533 of 60000
Storing image 59534 of 60000
Storing image 59535 of 60000
Storing image 59536 of 60000
Storing image 59537 of 60000
Storing image 59538 of 60000
Storing image 59539 of 60000
Storing image 59540 of 60000
Storing image 59541 of 60000
Storing image 59542 of 60000
Storing image 59543 of 60000
Storing image 59544 of 60000
Storing image 59545 of 60000
Storing image 59546 of 60000
Storing image 59547 of 60000
Storing image 59548 of 60000
Storing image 59549 of 60000
Storing image 59550 of 60000
Storing image 59551 of 60000
Storing image 59552 of 60000
Storing image 59553 of 60000
Storing image 59554 of 60000
Storing image 59555 of 60000
Storing image 59556 of 60000
Storing image 59557 of 60000
Storing image 59558 of 60000
Storing image 59559 of 60000
Storing image 59560 of 60000
Storing image 59561 of 60000
Storing image 59562 of 60000
Storing image 59563 of 60000
Storing image 

Storing image 59908 of 60000
Storing image 59909 of 60000
Storing image 59910 of 60000
Storing image 59911 of 60000
Storing image 59912 of 60000
Storing image 59913 of 60000
Storing image 59914 of 60000
Storing image 59915 of 60000
Storing image 59916 of 60000
Storing image 59917 of 60000
Storing image 59918 of 60000
Storing image 59919 of 60000
Storing image 59920 of 60000
Storing image 59921 of 60000
Storing image 59922 of 60000
Storing image 59923 of 60000
Storing image 59924 of 60000
Storing image 59925 of 60000
Storing image 59926 of 60000
Storing image 59927 of 60000
Storing image 59928 of 60000
Storing image 59929 of 60000
Storing image 59930 of 60000
Storing image 59931 of 60000
Storing image 59932 of 60000
Storing image 59933 of 60000
Storing image 59934 of 60000
Storing image 59935 of 60000
Storing image 59936 of 60000
Storing image 59937 of 60000
Storing image 59938 of 60000
Storing image 59939 of 60000
Storing image 59940 of 60000
Storing image 59941 of 60000
Storing image 

This section bit of code downloads the validation set for the MNIST dataset. It contains less images then before and should download a lot faster.

In [ ]:
validation_image_url = 'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz'
validation_label_url = 'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
validation_samples = 10000

download_dataset('mnist_test', validation_image_url, validation_label_url, validation_samples)

That's it, you now have all the data you need to run the samples!